In [4]:
import sys
import csv
import os
import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import sexmachine.detector as gender
from sklearn.preprocessing import Imputer
from sklearn import cross_validation
from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.cross_validation import StratifiedKFold,train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.learning_curve import learning_curve
from sklearn.metrics import roc_curve,auc,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report%matplotlib in line
frompybrain.structure import SigmoidLayer
frompybrain.datasets import ClassificationDataSet
frompybrain.utilities import percentError
frompybrain.tools.shortcuts import buildNetwork
frompybrain.supervised.trainers import BackpropTrainer
frompybrain.structure.modules import SoftmaxLayer
frompybrain.tools.xml.networkwriter import NetworkWriter
frompybrain.tools.xml.networkreader import NetworkReader
def read_datasets():
    genuine_users=pd.read_csv("data/users.csv")
    fake_users=pd.read_csv("data/fusers.csv")
    #printgenuine_users.columns
    #printgenuine_users.describe()
    #printfake_users.describe()
    x=pd.concat([genuine_users,fake_users])
    y=len(fake_users)*[0]+len(genuine_users)*[1]
    return x,y
def predict_sex(name):
    sex_predictor=gender.Detector(unknown_value=u"unknown",case_sensitive=False)
    first_name=name.str.split('').str.get(0)
    sex=first_name.apply(sex_predictor.get_gender)
    sex_dict={'female':-2,'mostly_female':-1,'unknown':0,'mostly_male':1,'male':2}
    sex_code=sex.map(sex_dict).astype(int)
    return sex_code
def extract_features(x):
    lang_list=list(enumerate(np.unique(x['lang'])))
    lang_dict={name:i for i,name in lang_list}
    x.loc[:,'lang_code']=x['lang'].map(lambdax:lang_dict[x]).astype(int)x.loc[:,'sex_code']=predict_sex(x['name'])
    feature_columns_to_use=['statuses_count','followers_count','friends_count','favourites_count','listed_count','sex_code','lang_code']
    x=x.loc[:,feature_columns_to_use]
    return x
def plot_confusion_matrix(cm,title='Confusionmatrix',cmap=plt.cm.Blues):
    target_names=['Fake','Genuine']
    plt.imshow(cm,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(target_names))
    plt.xticks(tick_marks,target_names,rotation=45)
    plt.yticks(tick_marks,target_names)
    plt.tight_layout()
    plt.ylabel('Truelabel')
    plt.xlabel('Predictedlabel')
def plot_roc_curve(y_test,y_pred):
    false_positive_rate,true_positive_rate,thresholds=roc_curve(y_test,y_pred)
    print"FalsePositiverate:",false_positive_rate
    print"TruePositiverate:",true_positive_rate
    roc_auc=auc(false_positive_rate,true_positive_rate)
    plt.title('ReceiverOperatingCharacteristic')
    plt.plot(false_positive_rate,true_positive_rate,'b',
    label='AUC=%0.2f'%roc_auc)
    plt.legend(loc='lowerright')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1,1.2])
    plt.ylim([-0.1,1.2])
    plt.ylabel('TruePositiveRate')
    plt.xlabel('FalsePositiveRate')
    plt.show()
def train(X,y):
    ds=ClassificationDataSet(len(X.columns),1,nb_classes=2)
    for k in x range(len(X)):
    ds.addSample(X.iloc[k],np.array(y[k]))
    tstdata,trndata=ds.splitWithProportion(0.20)
    trndata._convertToOneOfMany()
    tstdata._convertToOneOfMany()
    input_size=len(X.columns)
    target_size=1
    hidden_size=5
    fnn=None
    if os.path.isfile('fnn.xml'):
        fnn=NetworkReader.readFrom('fnn.xml')
    else:
        fnn=buildNetwork(trndata.indim,hidden_size,trndata.outdim,outclass=SoftmaxLayer)
        trainer=BackpropTrainer(fnn,dataset=trndata,momentum=0.05,learningrate=0.1,verbose=False,weightdecay=0.01)
    trainer.trainUntilConvergence(verbose=False,validationProportion=0.15,maxEpochs=100,continueEpochs=10)
    NetworkWriter.writeToFile(fnn,'oliv.xml')
    predictions=trainer.testOnClassData(dataset=tstdata)
    return tstdata['class'],predictions
print"readingdatasets.....\n"
x,y=read_datasets()
x.describe()
print"extractingfeatues.....\n"
x=extract_features(x)
printx.columns
printx.describe()
print"trainingdatasets.......\n"
y_test,y_pred=train(x,y)
print'ClassificationAccuracyonTestdataset:',accuracy_score(y_test,y_pred)
print'PercentErroronTestdataset:',percentError(y_pred,y_test)
cm=confusion_matrix(y_test,y_pred)
print('Confusionmatrix,withoutnormalization')
print(cm)
plot_confusion_matrix(cm)
cm_normalized=cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
print('Normalizedconfusionmatrix')
print(cm_normalized)
plot_confusion_matrix(cm_normalized,title='Normalizedconfusionmatrix')
print(classification_report(y_test,y_pred,target_names=['Fake','Genuine']))
s=roc_auc_score(y_test,y_pred)
print"roc_auc_score:",s
plot_roc_curve(y_test,y_pred)
def plot_learning_curve(estimator,title,X,y,ylim=None,cv=None,n_jobs=1,train_sizes=np.linspace(.1,1.0,5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
        plt.xlabel("Trainingexamples")
        plt.ylabel("Score")
        train_sizes,train_scores,test_scores=learning_curve(estimator,X,y,cv=cv,n_jobs=n_jobs,train_sizes=train_sizes)
        train_scores_mean=np.mean(train_scores,axis=1)
        train_scores_std=np.std(train_scores,axis=1)
        test_scores_mean=np.mean(test_scores,axis=1)
        test_scores_std=np.std(test_scores,axis=1)
        plt.grid()
        plt.fill_between(train_sizes,train_scores_mean-train_scores_std,
        train_scores_mean+train_scores_std,alpha=0.1,color="r")
        plt.fill_between(train_sizes,test_scores_mean-test_scores_std,
        test_scores_mean+test_scores_std,alpha=0.1,color="g")
        plt.plot(train_sizes,train_scores_mean,'o-',color="r",label="Trainingscore")
        plt.plot(train_sizes,test_scores_mean,'o-',color="g",label="Cross-validationscore")
        plt.legend(loc="best")
        return plt
def train(X_train,y_train,X_test):
    """TrainsandpredictsdatasetwithaRandomForestclassifier"""
    clf=RandomForestClassifier(n_estimators=40,oob_score=True)
    clf.fit(X_train,y_train)
    print("Thebestclassifieris:",clf)
    #Estimatescore
    scores=cross_validation.cross_val_score(clf,X_train,y_train,cv=5)
    printscores
    print('Estimatedscore:%0.5f(+/-%0.5f)'%(scores.mean(),scores.std()/2))
    title='LearningCurves(RandomForest)'
    plot_learning_curve(clf,title,X_train,y_train,cv=5)
    plt.show()
    #Predict
    y_pred=clf.predict(X_test)
    return y_test,y_pred
print "spliting datasets in train and test dataset...\n"
X_train,X_test,y_train,y_test = train_test_split(x, y, test_size=0.20, random_state=44)
print "training datasets.......\n"
y_test,y_pred = train(X_train,y_train,X_test)
print 'Classification Accuracy on Test dataset: ' ,accuracy_score(y_test, y_pred)
cm=confusion_matrix(y_test, y_pred)
print('Confusion matrix, without normalization')
print(cm)
plot_confusion_matrix(cm)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')


SyntaxError: invalid syntax (2960581394.py, line 53)